## Create a database connection

Storing data in the database instead instead of individual files would make it easier to manage multiple models and their versions.

In [1]:
# from pyswap.core.database import connection as con

# db_path = Path(pyswap_path, "pyswapdb.sqlite")
# con.DatabaseConnection(db_path)


## Metadata

Metadata is an important part of a project. Attached to the model definition provides some basic information about who created the model and how to contact them.

In [2]:
from pyswap.core.metadata import Metadata

meta = Metadata(author="John Doe",
                institution="University of Somewhere",
                email="john.doe@somewhere.com",
                project="Test",
                swap_ver="4.0")

## Simulation settings

The first thing to do is to define the simulation settings. The `SimSettings` object will store the information about the environment, files the user wants to output, simulation period, etc. 

In [3]:
from pyswap.core.simsettings import SimSettings
from datetime import date as dt

simset = SimSettings(tstart='2018-01-01', 
                     swyrvar=1,
                     tend=dt(2018, 1, 2),
                     outdat=[dt(2018, 1, 1), dt(2018, 1, 1)],
                     inlist_csv=['rain', 'evap'],
                     dznew=[0.13452, 0.2345, 0.333])


## Meteorology

Usually a SWAP model requires some form of meteorological data in the form of .met file. For simplicity, we use a csv file (with .met extension) containing data from multiple years. One of the implemented methods fetches the data from KNMI API.

In [4]:
from pyswap.atmosphere.meteorology import MeteorologicalData

meteo_data = MeteorologicalData()
meteo_data.weather_kmni(station='377')

In [5]:
from pyswap.atmosphere.meteorology import PenmanMonteith
pen_mon = PenmanMonteith(alt=100.0)

In [6]:
from pyswap.atmosphere.meteorology import Meteorology, PenmanMonteith
from pandas import DataFrame

rainflux = DataFrame({'time': [1.0, 360.0], 
            'rainflux': [20.0, 20.0]})

meteo = Meteorology(metfil='377',
                    file_meteo=meteo_data,
                    lat=43.0,
                    swetr=1,
                    penman_monteith=pen_mon,
                    swdivide=0,
                    swetsine=0,
                    swrain=0,
                    table_rainflux=rainflux)



## Crop

In [9]:
from pyswap.plant.crop import CropFile

maizes = CropFile(name='maizes', path='../../data/.crp/maizes.crp')
grassd = CropFile(name='grassd', path='../../data/.crp/grassd.crp')
potatod = CropFile(name='potatod', path='../../data/.crp/potatod.crp')


In [10]:
from pyswap.plant.crop import Crop
from pandas import DataFrame
from datetime import date as dt
# Would be nice to have a subclass for managing the crop files

croprotation = DataFrame({'cropstart':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropend':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropfile':['maizes', 'potatod', 'grassd'],
                'croptype':[1, 2, 3]})


crop = Crop(swcrop=1,
            rds=200.0,
            table_croprotation=croprotation,
            cropfiles=[maizes, grassd, potatod])

In [11]:
crop.save_crop('./')

'3 crop file(s) saved successfully.'

## Irrigation

In [1]:
from pyswap.soilwater.irrigation import Irrigation, FixedIrrigation
from pandas import DataFrame

fixed_irrigation = FixedIrrigation(swirgfil=0,
                                   table_irrigevents=DataFrame({'time':[1.0, 360.0],
                                                            'irrigation':[20.0, 20.0]}))

irrigation = Irrigation(swirfix=1,
                        fixedirrig=fixed_irrigation,
                        schedule=0)
                                   

## Drainage

In [2]:
from pyswap.soilwater.drainage import LateralDrainage, DrainageFile
dra_file = DrainageFile(name='swap', path='../../data/swap.dra')
latdrain = LateralDrainage(swdra=0, drainagefile=dra_file)

In [3]:
latdrain

LateralDrainage(swdra=0, drfil=None, drainagefile=DrainageFile(name='swap', path='../../data/swap.dra', content='**********************************************************************************\n* Filename: swap.dra                  \n* Contents: SWAP 4 - Input data for basic drainage and extended drainage\n**********************************************************************************\n* Comment area:\n* Testbank of SWAP: build with template of dra-file\n*\n**********************************************************************************\n\n*** BASIC DRAINAGE SECTION ***\n\n**********************************************************************************\n* Part 0: General\n\n* Switch, method of lateral drainage calculation:\n  DRAMET = 2                 ! 1 = Use table of drainage flux - groundwater level relation\n                             ! 2 = Use drainage formula of Hooghoudt or Ernst\n                             ! 3 = Use drainage/infiltration resistance, multi-level 

In [2]:
latdrain.save_drainage('./')

'.dra file saved successfully.'

## Soil water

In [12]:
from pyswap.soilwater.soilmoisture import SoilMoisture
from pyswap.soilwater.surfaceflow import SurfaceFlow
from pyswap.soilwater.evaporation import Evaporation


soilmoisture = SoilMoisture(swinco=2,
                            gwli=-75.0)

surfaceflow = SurfaceFlow(swpondmx=0,
                          pondmx=0.2,
                          swrunon=0)

evaporation = Evaporation(swcfbs=0,
                          cfevappond=1.25,
                          swredu=1,
                          cofredbl=0.35,
                          rsigni=0.5)


In [1]:
from pyswap.soilwater.soilprofile import SoilProfile
from pandas import DataFrame

df_soilprofile = DataFrame({
    'ISUBLAY': [1, 2, 3, 4],
    'ISOILLAY': [1, 1, 2, 2],
    'HSUBLAY': [10.0, 20.0, 30.0, 140.0],
    'HCOMP': [1.0, 5.0, 5.0, 10.0],
    'NCOMP': [10, 4, 6, 14]
})

soilprofile = SoilProfile(table_soilprofile=df_soilprofile,
                          swsophy=0,
                          swhyst=0,
                          swmacro=0)



In [2]:
soilprofile

SoilProfile(table_soilprofile=   ISUBLAY  ISOILLAY  HSUBLAY  HCOMP  NCOMP
0        1         1     10.0    1.0     10
1        2         1     20.0    5.0      4
2        3         2     30.0    5.0      6
3        4         2    140.0   10.0     14, swsophy=0, swhyst=0, swmacro=0, filenamesophy=(None,), other_str=(None,), tau=None, table_soilhydrfunc=None)

In [14]:
from pyswap.soilwater.snow import SnowAndFrost

snow = SnowAndFrost(swsnow=0,
                    swfrost=0)


In [15]:
from pyswap.soilwater.richards import RichardsSettings

richards = RichardsSettings(swkmean=1,
                            swkimpl=0)


In [16]:
from pyswap.core.boundary import BottomBoundary

bottom_boundary = BottomBoundary(swbbcfile=0, 
                                 swbotb=6)

## Putting the model together

In [18]:
from pyswap.core.model import Model


model = Model(metadata=meta,
            simsettings=simset,
            meteorology=meteo,
            crop=crop,
            irrigation=irrigation,
            soilmoisture=soilmoisture,
            surfaceflow=surfaceflow,
            evaporation=evaporation,
            soilprofile=soilprofile,
            snowandfrost=snow,
            richards=richards,
            lateraldrainage=latdrain,
            bottomboundary=bottom_boundary)

In [19]:
string = model.concat_swp()

/home/zawadzkim/miniconda3/envs/pyswap/lib/python3.11/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [20]:
print(string)

PROJECT = "Test"
PATHWORK = "/home/zawadzkim/projects/pySWAP/docs/notebooks"
PATHATM = "/home/zawadzkim/projects/pySWAP/docs/notebooks"
PATHCROP = "/home/zawadzkim/projects/pySWAP/docs/notebooks"
PATHDRAIN = "/home/zawadzkim/projects/pySWAP/docs/notebooks"
SWSCRE = 0
SWERROR = 1
TSTART = 2018-01-01
TEND = 2018-01-02
NPRINTDAY = 1
SWMONTH = 1
SWYRVAR = 1
OUTDAT = 
2018-01-01
2018-01-01
OUTFIL = "result"
SWHEADER = 0
SWWBA = 0
SWEND = 0
SWVAP = 0
SWBAL = 0
SWBLC = 0
SWSBA = 0
SWATE = 0
SWBMA = 0
SWDRF = 0
SWSWB = 0
SWINI = 0
SWINC = 0
SWCRP = 0
SWSTR = 0
SWIRG = 0
SWCSV = 1
INLIST_CSV = "rain,evap"
SWCSV_TZ = 0
SWAFO = 0
SWAUN = 0
SWDISCRVERT = 0
DZNEW = 0.13452 0.2345 0.333
METFIL = 377
LAT = 43.0
SWETR = 1
SWDIVIDE = 0
SWETSINE = 0
ALT = 100.0
ALTW = 10.0
ANGSTROMA = 0.25
ANGSTROMB = 0.5
SWRAIN = 0
 TIME  RAINFLUX
  1.0      20.0
360.0      20.0
SWCROP = "True"
RDS = 200.0
 CROPSTART    CROPEND CROPFILE  CROPTYPE
2018-01-01 2018-01-01   maizes         1
2018-01-01 2018-01-01  potatod  